<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor any WML online deployment as a self managed subscription on Openscale. For self managed subscriptions, all the monitors are run as jobs on user provided spark engine. Use the notebook to enable quality, drift, drift v2, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. Configuration Archive generated using common configuration notebook [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb) and pre created WML deployment.
2. Payload, feedback, drifted transactions, explanations queue and result tables to be created or validated in a supported database (DB2 or Postgres) storage. Please note, some of these maybe optional depending on your use-case.

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up Subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Drift V2 monitoring](#drift_v2)
* [7. Fairness monitoring](#fairness)
* [8. Explainability monitoring](#explainability)

## Setup <a name="setup"></a>

### Installing Required Libraries

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

### Import configuration archive/package

Configuration archive/package created using configuration notebook will be required to onboard model for monitoring in IBM Watson OpenScale. Provide path location of archive here.

Please note if you are executing this notebook in IBM Watson Studio, first upload the configuration archive/package to project and use provided code snippet to download it to local directory of this notebook.

In [1]:
import warnings
import tarfile
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

# Note: Restart kernel after the dependencies are installed
!pip install --upgrade ibm-watson-openscale
!pip install "ibm_wos_utils~=5.1.1"

# # Download "configuration_archive.tar.gz" from project to local directory and 
# extract the artifacts
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()
# wslib.download_file("configuration_archive.tar.gz")
archive_file_path = "configuration_archive.tar.gz"
with tarfile.open('configuration_archive.tar.gz', "r:gz") as tar:
    tar.extractall('.')
# On unpacking the configuration archive, the files [common_configuration.json, drift_archive.tar.gz, drift_v2_archive.tar.gz,
# explainability.tar.gz, fairness_statistics.json] will be extracted to the current directory. 
# Output may differ depending on which monitors were enabled while running the 
# common configuration notebook

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [2]:
!pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.42
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [3]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

WML_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [4]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [5]:
IAE_SPARK_DISPLAY_NAME = ""
IAE_SPARK_JOBS_ENDPOINT = ""
IBM_CPD_VOLUME = ""
IBM_CPD_USERNAME = ""
IBM_CPD_APIKEY = ""
IAE_SPARK_NAME = ""
IAE_SPARK_DESCRIPTION = ""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [6]:
spark_parameters = {
    "max_num_executors": 1,
    "min_num_executors": 1,
    "executor_cores": 1,
    "executor_memory": 1,
    "driver_cores": 1,
    "driver_memory": 1
}

### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.
 - JDBC_CONNECTION_TYPE: Type of the JDBC Connection.

In [7]:
JDBC_CONNECTION_NAME = ""
JDBC_CONNECTION_DESCRIPTION = ""

JDBC_CONNECTION_TYPE = "" # supported: db2, postgresql
JDBC_HOST = ""
JDBC_PORT = ""
JDBC_USE_SSL = ""
JDBC_SSL_CERTIFICATE = ""
JDBC_DRIVER = ""
JDBC_USERNAME = ""
JDBC_PASSWORD = ""
JDBC_DATABASE_NAME = ""

JDBC_CONNECTION_NAME = "jdbc for batch parity"
JDBC_CONNECTION_DESCRIPTION = "jdbc connection from SDK

In [8]:
num_partitions_recommended = 12

if spark_parameters:
    executors = spark_parameters.get("max_num_executors", 2)
    cores = spark_parameters.get("executor_cores", 2)
    num_partitions_recommended = 2 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

2 is the recommended value for number of partitions in your data. Please change this value as per your data.


- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`.
- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [9]:
PARTITION_COLUMN = "wos_partition_column"
NUM_PARTITIONS = num_partitions_recommended

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [10]:
#feedback

FEEDBACK_DATABASE_NAME = None
FEEDBACK_SCHEMA_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [11]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_SCHEMA_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_SCHEMA_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [12]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_SCHEMA_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [13]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])
data_mart_id=WOS_CREDENTIALS["instance_id"]

### Display Watson OpenScale datamart details

In [14]:
wos_client.data_marts.show()

### Create a service provider

In [15]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [16]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        credentials=WML_CREDENTIALS,
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider c30290f1-cf79-4067-b4ea-66794f93f0f3 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




Note: First 10 records were displayed.


In [17]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

{
  "metadata": {
    "id": "c30290f1-cf79-4067-b4ea-66794f93f0f3",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:c30290f1-cf79-4067-b4ea-66794f93f0f3",
    "url": "/v2/service_providers/c30290f1-cf79-4067-b4ea-66794f93f0f3",
    "created_at": "2025-01-27T08:45:27.225000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML-DriftV2Batch",
    "service_type": "watson_machine_learning",
    "instance_id": "00000000-0000-0000-0000-000000000000",
    "credentials": {
      "secret_id": "ae8c6a1a-e778-40d6-a060-72a80291f269"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


### Create integrated systems for Spark Engine and JDBC Storage

In [18]:
# Delete existing spark and jdbc integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, JDBC_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system jdbc for batch parity
Deleting integrated system IAEBatchSpark
Deleting integrated system IAEBatchSpark


#### Spark Engine

In [19]:
spark_integrated_system = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_integrated_system_id = spark_integrated_system.metadata.id
print(spark_integrated_system)

{
  "metadata": {
    "id": "07b876b5-abb3-4bd1-9231-64478674ce44",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:07b876b5-abb3-4bd1-9231-64478674ce44",
    "url": "/v2/integrated_systems/07b876b5-abb3-4bd1-9231-64478674ce44",
    "created_at": "2025-01-27T08:45:37.469000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "IAEBatchSpark",
    "type": "spark",
    "description": "",
    "credentials": {
      "secret_id": "9afa4847-fbb3-4f33-96bb-b06fba6ef8dd"
    },
    "connection": {
      "display_name": "IAEBatchSpark",
      "endpoint": "https://cpd-cpd-instance.apps.osdev510pwx417gmc-1.cp.fyre.ibm.com/v4/analytics_engines/74dc7110-8603-4c64-961d-bdbb44296cfd/spark_applications",
      "location_type": "cpd_iae",
      "volume": "cpd-instance::IAEBatchTest"
    }
  }
}


#### JDBC Storage

In [20]:
jdbc_url = "jdbc:{}://{}:{}/{}".format(JDBC_CONNECTION_TYPE, JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

jdbc_integrated_system = IntegratedSystems(wos_client).add(
    name=JDBC_CONNECTION_NAME,
    description=JDBC_CONNECTION_DESCRIPTION,
    type="jdbc",
    credentials={
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
        "jdbc_url": jdbc_url
    },
    connection={
        "location_type": "jdbc",
        "db_driver": JDBC_DRIVER,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
    }
).result

jdbc_integrated_system_id=jdbc_integrated_system.metadata.id
print(jdbc_integrated_system)

{
  "metadata": {
    "id": "ab80a0c9-eed1-4540-a08c-309579a32304",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:ab80a0c9-eed1-4540-a08c-309579a32304",
    "url": "/v2/integrated_systems/ab80a0c9-eed1-4540-a08c-309579a32304",
    "created_at": "2025-01-27T08:45:38.166000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "jdbc for batch parity",
    "type": "jdbc",
    "description": "jdbc connection from SDK",
    "credentials": {
      "secret_id": "3328361a-75dd-47a5-884e-7cc957fc4f6e"
    },
    "connection": {
      "certificate": "",
      "db_driver": "org.postgresql.Driver",
      "location_type": "jdbc",
      "use_ssl": false
    }
  }
}


# 3. Set up a subscription<a name="subscription"></a>

In [21]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, we expect you extracted Configuration Archive to current directory and it contains common_configuration.json. If not true, please update path to correct location. After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [22]:
import uuid
import json
# Provide the WML deployment io and space d
deployment_uid = ""
space_id = ""

common_configuration = None
with open("common_configuration.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        raise Exception("Please provide the correct path to the common configuration JSON")
        
asset_deployment_details = wos_client.service_providers.list_assets(
    data_mart_id=data_mart_id, service_provider_id=service_provider_id, 
    deployment_id = deployment_uid, deployment_space_id = space_id).result['resources'][0]

model_asset_details_from_deployment = wos_client.service_providers.get_deployment_asset(
    data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,
    deployment_space_id=space_id)

scoring_endpoint = ScoringEndpointRequest(
    url=model_asset_details_from_deployment['entity']['scoring_endpoint']['url'] )

    
# Set asset details
asset = Asset(
    asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
    name=model_asset_details_from_deployment["entity"]["asset"]["name"],
    url=model_asset_details_from_deployment["entity"]["asset"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=common_configuration.get("problem_type") if common_configuration.get("problem_type") else common_configuration.get("model_type")
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=asset_deployment_details['metadata']['guid'],
    name=asset_deployment_details['entity']['name'],
    deployment_type= DeploymentTypes.ONLINE,
    url=asset_deployment_details['entity']['scoring_endpoint']['url'],
    scoring_endpoint=scoring_endpoint
)

probability_fields = common_configuration.get("probability_fields") if common_configuration.get("probability_fields") else [common_configuration["probability"]]

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=probability_fields, # comment out this line for regression models as probability_fields is not applicable
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"],
    training_data_schema=SparkStruct.from_dict(common_configuration["training_data_schema"]),
    input_data_schema=SparkStruct.from_dict(common_configuration["input_data_schema"]),
    output_data_schema=SparkStruct.from_dict(common_configuration["output_data_schema"])
)

In [23]:
# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_integrated_system_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME,
        schema_name=FEEDBACK_SCHEMA_NAME,
        table_name=FEEDBACK_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME,
        schema_name=PAYLOAD_SCHEMA_NAME,
        table_name=PAYLOAD_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME,
        schema_name=DRIFT_SCHEMA_NAME,
        table_name=DRIFT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:
    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME, 
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [24]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    scoring_endpoint=scoring_endpoint).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "52565726-9e31-43aa-bd54-64690ab9d272",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:52565726-9e31-43aa-bd54-64690ab9d272",
    "url": "/v2/subscriptions/52565726-9e31-43aa-bd54-64690ab9d272",
    "created_at": "2025-01-27T08:45:43.624000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "c30290f1-cf79-4067-b4ea-66794f93f0f3",
    "asset": {
      "asset_id": "fcd58b54-c281-449c-938f-8c20632d7671",
      "url": "https://internal-nginx-svc:12443/ml/v4/models/fcd58b54-c281-449c-938f-8c20632d7671?space_id=0bd7e44c-bdea-4c1f-a4b2-a32971945f05&version=2020-06-12",
      "name": "GermanCreditRiskModelICP",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "training_data_schema": {
        "type": "struct",
        "fields": [
  

In [25]:
import time
subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
        
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

14:15:44 preparing
14:16:00 active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [26]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 1000
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "6bc855ca-b1c3-40f1-9e47-9f663754272e",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:6bc855ca-b1c3-40f1-9e47-9f663754272e",
    "url": "/v2/monitor_instances/6bc855ca-b1c3-40f1-9e47-9f663754272e",
    "created_at": "2025-01-27T08:46:05.697000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [28]:
from datetime import datetime

quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_status)

14:18:55 preparing
14:19:26 preparing
14:19:57 preparing
14:20:27 preparing
14:20:59 active


In [29]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "6bc855ca-b1c3-40f1-9e47-9f663754272e",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:6bc855ca-b1c3-40f1-9e47-9f663754272e",
    "url": "/v2/monitor_instances/6bc855ca-b1c3-40f1-9e47-9f663754272e",
    "created_at": "2025-01-27T08:46:05.697000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-27T08:50:43.187000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "job_validation_schedule_id": "fe370426-af6d-46de-945f-82d3007a7a16",
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat

### Run an on-demand evaluation

Please make sure you have data in feedback table. Quality evaluation will use this data for its metrics computation.

In [30]:
# Trigger on-demand run
monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "a48c2abc-132a-4b86-8b8c-90d189e4a9d0",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:a48c2abc-132a-4b86-8b8c-90d189e4a9d0",
    "url": "/v2/monitor_instances/6bc855ca-b1c3-40f1-9e47-9f663754272e/runs/a48c2abc-132a-4b86-8b8c-90d189e4a9d0",
    "created_at": "2025-01-27T08:51:00.696000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "job_validation_schedule_id": "fe370426-af6d-46de-945f-82d3007a7a16",
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2025-01-27T08:51:00.688000Z",
      "started_at": "2025-01-27T08:51:00.696000Z",
      "operators": []
    }
  }
}


In [31]:
# Check run status
quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
print(datetime.now().strftime("%H:%M:%S"), quality_run_status)

14:21:01 running
14:21:32 running
14:22:02 running
14:22:34 running
14:23:05 running
14:23:35 running
14:24:06 running
14:24:39 running
14:25:09 running
14:25:40 running
14:26:11 finished


### Display quality metrics

In [32]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [34]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': 'eb7b77e5-29ab-4ac9-a60e-92a0fbfc2acc',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 1000,
    'appr

In [35]:
# Delete existing drift monitor.
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of minimum records required in an evaluation time window for drift analysis to be completed. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to learn drift artefacts (drift model training and data constraints learning) by IBM Watson OpenScale via Spark job accessing training data table.

In [36]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 1000,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "c6ef1670-edbd-45c8-b7b6-9a1178e71456",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:c6ef1670-edbd-45c8-b7b6-9a1178e71456",
    "url": "/v2/monitor_instances/c6ef1670-edbd-45c8-b7b6-9a1178e71456",
    "created_at": "2025-01-27T08:57:46.143000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 10,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_dr

### Check monitor instance status

In [38]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_status)
        time.sleep(60)

print(datetime.now().strftime("%H:%M:%S"), drift_status)

14:28:33 preparing
14:29:33 preparing
14:30:34 active


### Run an on-demand evaluation
Please make sure you have data in payload table. Drift evaluation will use this data for its metrics computation.

In [39]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "c6ef1670-edbd-45c8-b7b6-9a1178e71456",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:c6ef1670-edbd-45c8-b7b6-9a1178e71456",
    "url": "/v2/monitor_instances/c6ef1670-edbd-45c8-b7b6-9a1178e71456",
    "created_at": "2025-01-27T08:57:46.143000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-27T09:00:31.653000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0

In [40]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "74feace0-c26b-461d-8f44-98181f15b683",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:74feace0-c26b-461d-8f44-98181f15b683",
    "url": "/v2/monitor_instances/c6ef1670-edbd-45c8-b7b6-9a1178e71456/runs/74feace0-c26b-461d-8f44-98181f15b683",
    "created_at": "2025-01-27T09:00:35.038000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0.05,
      "min_samples": 10,
      "model_drift_enabled": true,
      "table_schema": {
        "fields": [
          {
            "length": 64,
            "metadata": {},
            "name": "scoring_id",
            "nullable": false,
        

In [41]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
print(datetime.now().strftime("%H:%M:%S"), drift_run_status)

14:30:35 running
14:31:06 running
14:31:36 running
14:32:07 running
14:32:37 running
14:33:14 finished


### Display drift metrics

In [42]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

# 6. Drift V2 monitoring <a name="drift_v2"></a>

### Enable the drift v2 monitor

In the following code cell, type a path to the drift v2 configuration tar ball.

In [43]:
# Upload drift_v2 archive

wos_client.monitor_instances.upload_drift_v2_archive(
    archive_path="drift_v2_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{}

In [44]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift v2 monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing drift v2 monitor instance with id:  27059f1b-ffe7-462d-bdf4-6f47b5cb79f8


In [45]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 100,
        "max_samples": 1000,
        "train_archive": False,
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'2ebdceff-545c-40dc-815e-a77084a610c3'

### Check monitor instance status

In [46]:
drift_v2_status = None

while drift_v2_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
    drift_v2_status = monitor_instance_details.entity.status.state
    if drift_v2_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)

16:22:16 preparing
16:22:46 active


In [47]:
# Check Drift v2 monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "2ebdceff-545c-40dc-815e-a77084a610c3",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:2ebdceff-545c-40dc-815e-a77084a610c3",
    "url": "/v2/monitor_instances/2ebdceff-545c-40dc-815e-a77084a610c3",
    "created_at": "2025-01-27T10:52:12.554000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-27T10:52:31.344000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
   

In [48]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "a9483c5f-b67e-47e2-86d0-d2ccc877b770",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:a9483c5f-b67e-47e2-86d0-d2ccc877b770",
    "url": "/v2/monitor_instances/2ebdceff-545c-40dc-815e-a77084a610c3/runs/a9483c5f-b67e-47e2-86d0-d2ccc877b770",
    "created_at": "2025-01-27T11:34:28.976000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
        },
        "data_sets": {
          "drift_insights": "8be25a14-4058-43e8-ad34-3cc25675d625",
          "drift_intervals": "62c8a8df-a741-497a-a7f0-3a7fb9e3f5f1",
          "drift_scores": "c4be141f-dd7a-445b-95fb-499f45ee1e49",
          "drift_

In [49]:
# Check run status
drift_v2_run_status = None
while drift_v2_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_v2_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_v2_run_status = monitoring_run_details.entity.status.state
    if drift_v2_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)



17:04:32 running
17:05:03 running
17:05:35 running
17:06:05 running
17:06:37 running
17:07:08 running
17:07:39 running
17:08:10 running
17:08:41 running
17:09:12 running
17:09:43 running
17:10:14 running
17:10:45 running
17:11:15 running
17:11:46 running
17:12:17 running
17:12:48 running
17:13:18 running
17:13:49 running
17:14:19 running
17:14:50 running
17:15:21 running
17:15:51 running
17:16:22 running
17:16:53 running
17:17:24 running
17:17:57 running
17:18:27 running
17:18:58 running
17:19:28 finished


### Check Drift V2 metrics

In [50]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

Note: First 10 records were displayed.


# 7. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [51]:
#Delete existing monitor instance

monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "fairness" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing fairness monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing fairness monitor instance with id:  b236ae94-d697-4615-87b3-dc2c109ecdd1


In [52]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

fairness_statistics_json = None
with open("fairness_statistics.json", "r") as fp:
    fairness_statistics_json = json.load(fp)
    
parameters = fairness_statistics_json["parameters"]
thresholds = fairness_statistics_json["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "593c0702-d4fb-4628-afb9-1c92acf74577",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:593c0702-d4fb-4628-afb9-1c92acf74577",
    "url": "/v2/monitor_instances/593c0702-d4fb-4628-afb9-1c92acf74577",
    "created_at": "2025-01-29T07:04:09.345000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "training_data_last_processed_time": "2025-01-17T09:40:55.219953Z",
      "min_records": 100,
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "av

### Check monitor instance status

In [53]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

07:05:28 preparing
07:05:58 active


### Run an on-demand evaluation
Please make sure you have data in payload table. Fairness evaluation will use this data for its metrics computation.

In [54]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "acc03c19-45f6-447c-b536-312f604b5041",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:acc03c19-45f6-447c-b536-312f604b5041",
    "url": "/v2/monitor_instances/593c0702-d4fb-4628-afb9-1c92acf74577/runs/acc03c19-45f6-447c-b536-312f604b5041",
    "created_at": "2025-01-29T07:05:59.226000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "average_abs_odds_difference",
            "false_negative_rate_difference",
            "false_positive_rate_difference",
            "false_discovery_rate_difference",
            "false_omission_rate_differenc

In [55]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)   
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

07:05:59 running
07:06:30 running
07:07:00 running
07:07:30 running
07:08:01 running
07:08:31 running
07:09:02 running
07:09:32 running
07:10:03 running
07:10:33 running
07:11:03 running
07:11:34 finished


### Display fairness metrics

In [56]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

Note: First 10 records were displayed.


# 8. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [57]:
with open("explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

Uploaded explainability archive successfully.


In [58]:
#Delete existing monitor instance

monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "explainability" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing explainability monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing explainability monitor instance with id:  4a99f104-7c24-4bb8-b6d6-3c5d1d90ba3f


In [59]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
# Uncomment the below lines to enable lime global explanation
# Lime global explanation is available from Cloud Pak for Data version 4.6.4 onwards.
   "global_explanation": {
       "enabled": True,  # Flag to enable global explanation 
       "explanation_method": "lime",
    #    "sample_size": 10, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
   }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "161a2af3-fec0-444a-8434-00ddc993160c",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:161a2af3-fec0-444a-8434-00ddc993160c",
    "url": "/v2/monitor_instances/161a2af3-fec0-444a-8434-00ddc993160c",
    "created_at": "2025-01-29T07:37:41.532000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "global_explanation": {
        "enabled": false,
        "explanation_method": "lime"
      }
    },
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "status": {
      "state": "preparing"
    }
  }
}


### Check monitor instance status

In [60]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

07:40:09 preparing
07:40:40 preparing
07:41:10 preparing
07:41:41 preparing
07:42:11 preparing
07:42:41 preparing
07:43:12 active


### Run an on-demand evaluation
Please make sure you have data in Explain queue table. Explainability will use this data for computing explanations.

In [61]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "161a2af3-fec0-444a-8434-00ddc993160c",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:161a2af3-fec0-444a-8434-00ddc993160c",
    "url": "/v2/monitor_instances/161a2af3-fec0-444a-8434-00ddc993160c",
    "created_at": "2025-01-29T07:37:41.532000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-29T07:42:46.075000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "52565726-9e31-43aa-bd54-64690ab9d272"
    },
    "parameters": {
      "config_modified_at": "2025-01-29T07:42:43.030064Z",
      "config_package_file": "explainability.tar (2).gz",
      "contrastive": {
        "enabled": true
      },
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 

In [62]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "4961c912-3c7f-4b02-8743-6b421b8dfaec",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:4961c912-3c7f-4b02-8743-6b421b8dfaec",
    "url": "/v2/monitor_instances/161a2af3-fec0-444a-8434-00ddc993160c/runs/4961c912-3c7f-4b02-8743-6b421b8dfaec",
    "created_at": "2025-01-29T07:48:33.680000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_modified_at": "2025-01-29T07:42:43.030064Z",
      "config_package_file": "explainability.tar (2).gz",
      "contrastive": {
        "enabled": true
      },
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "global_explanation": {
        "aggregation_method": "mean_abs",
        "aggregation_methods": [
          "mean_abs"
        ],
        "baseline_explanation_measurement_id": "82d746d7-e4a0-46b7-9220-e0cabfc7fb9a",
        "distribution_

In [63]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
print(monitoring_run_details)

07:48:35 running
07:49:35 running
07:50:36 running
07:51:36 running
07:52:37 running
07:53:50 running
07:55:03 finished
{
  "metadata": {
    "id": "4961c912-3c7f-4b02-8743-6b421b8dfaec",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:4961c912-3c7f-4b02-8743-6b421b8dfaec",
    "url": "/v2/monitor_instances/161a2af3-fec0-444a-8434-00ddc993160c/runs/4961c912-3c7f-4b02-8743-6b421b8dfaec",
    "created_at": "2025-01-29T07:48:33.680000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "batch_job_app_id": null,
      "batch_job_data_path": "explanations_computation_job/14905c12-4f03-4d12-80e6-e073f5d1c21e/data",
      "batch_job_id": "acdc29e4-b3c9-4a79-8b52-5f188250ffa1",
      "batch_job_output_path": "explanations_computation_job/14905c12-4f03-4d12-80e6-e073f5d1c21e/output/4961c912-3c7f-4b02-8743-6b421b8dfaec",
      "batch_job_poll_schedule_id": "bc99faf6-b653-4dec-9341-e3a68df4dea

In [64]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [65]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result.to_dict()
print(explanations)

{'total_count': 50, 'limit': 50, 'offset': 0, 'explanation_fields': ['explanation_task_id', 'scoring_id', 'created_at', 'finished_at', 'status', 'prediction', 'subscription_id', 'deployment_id', 'asset_name', 'deployment_name', 'probability', 'explanation_type'], 'explanation_values': [['ac82f880-6f6a-4552-87f9-a5c27f2f99ee', '96927a161a0d60c579981e93f8d78fc2-7', '2025-01-29T07:49:49.247001Z', '2025-01-29T07:49:49.247160Z', 'finished', 'No Risk', '52565726-9e31-43aa-bd54-64690ab9d272', 'd31e0810-b1ad-457f-bc91-a5aedf40d26c', 'GermanCreditRiskModelICP', 'GermanCreditRiskModelICP', 0.677657, 'lime'], ['a2d8a1a1-4391-495f-bb4b-13ee5243c78d', '43e915f6d7b53a260171de2886d3d661-2', '2025-01-29T07:49:49.222008Z', '2025-01-29T07:49:49.222181Z', 'finished', 'No Risk', '52565726-9e31-43aa-bd54-64690ab9d272', 'd31e0810-b1ad-457f-bc91-a5aedf40d26c', 'GermanCreditRiskModelICP', 'GermanCreditRiskModelICP', 0.677657, 'lime'], ['ccd6c14d-1e7c-4e9e-8be0-d859ba099954', '9820ff2a017cadb6ada7931f23adf6f7-

### Generate on demand explanation

Choose any record from payload logging table or Explain Queue table and generate Lime, Contrastive explanations for the record. Please note that contrastive explanations are only supported for structured classification models. So specify the explanation_types accordingly. 

In [ ]:
import time
# Please enter scoring id(from payload table or Explain Queue table), for which you want to generate an explanation
# scoring_ids = ["<scoring_id_from_pl_or_explain_queue_table>"] 

# As an example, we will generate a contrastive explanation for the first scoring id from the list of explanations generated above
# as part of the Explainability monitor run
scoring_ids = [explanations["explanation_values"][0][1]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))

explanation_types = ["lime", "contrastive"]
explanation_task_ids = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, subscription_id=subscription_id, explanation_types=explanation_types).result.metadata.explanation_task_ids
explanation_task_id = explanation_task_ids[0]

print("Getting explanation for explanation task id: {}".format(explanation_task_id))
explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
while explanation.entity.status.state not in ("finished", "error"):
        explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
        explanation_run_status = explanation.entity.status.state
        if explanation_run_status not in ("finished", "error"):
            print(datetime.utcnow().strftime("%H:%M:%S"), explanation_run_status)
            time.sleep(60)
            
            
print(explanation)

### Cleanup the untarred files

In [ ]:
import os
files = ["common_configuration.json", "explainability.tar.gz", "drift_archive.tar.gz", "fairness_statistics.json"]
for file in files:
    if os.path.isfile(file):
        os.remove(file)

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the model you configured to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.